In [ ]:
! [ -e /content ] && pip install -Uqq fastai
from fastai.vision.all import *
import os

In [ ]:
path = '../results'
test = '../images/A_test_set'
gan_files = get_image_files('../results/gan_results')
lsgan_files = get_image_files('../results/lsgan_results')
wgan_files = get_image_files('../results/wgan_results')
print(len(gan_files), len(lsgan_files), len(wgan_files))

In [ ]:
def label_func(f): return f[0].isupper()
gan_dls = ImageDataLoaders.from_name_func(path, gan_files, label_func, item_tfms=Resize(128))
lsgan_dls = ImageDataLoaders.from_name_func(path, lsgan_files, label_func, item_tfms=Resize(128))
wgan_dls = ImageDataLoaders.from_name_func(path, wgan_files, label_func, item_tfms=Resize(128))

In [ ]:
gan_learn = vision_learner(gan_dls, resnet34, metrics=accuracy)
gan_learn.fine_tune(5)

In [ ]:
lsgan_learn = vision_learner(lsgan_dls, resnet34, metrics=accuracy)
lsgan_learn.fine_tune(5)

In [ ]:
wgan_learn = vision_learner(wgan_dls, resnet34, metrics=accuracy)
wgan_learn.fine_tune(5)

In [ ]:
def test_accuracy(learner):
  results = 0
  healthy = 0

  for image in os.scandir(test):
    if str(image)[11].isupper() and learner.predict(test + str(image)[11:-2])[0] == 'True':
      results += 1
      healthy += 1
    elif str(image)[11].islower() and learner.predict(test + str(image)[11:-2])[0] == 'False':
      results += 1

  print('Results for ' + learner)
  print(f'Overall accuracy: {results}%')
  print(f'Healthy accuracy: {healthy * 2}%')
  print(f'Rusted accuracy: {(results - healthy) * 2}%')

In [ ]:
test_accuracy(gan_learn)
test_accuracy(lsgan_learn)
test_accuracy(wgan_learn)